In [1]:
import os
import pandas as pd
import numpy as np
import PIL
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import model_selection
from PIL import Image
from skimage import exposure

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Datos

## Dataframe

In [3]:
path = '/home/maryskal/Downloads/archive/vinbigdata'
path = '/home/mr1142/Documents/Data/vinbigdata'

In [4]:
df = pd.read_csv(os.path.join(path, 'train.csv'))

In [5]:
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072


In [6]:
for i, finding in enumerate(np.unique(df.class_name)):
    print(f'class name: {finding}')
    print(f'ID: {df.class_id[i]}')
    print(sum(df.class_name == finding))

class name: Aortic enlargement
ID: 14
7162
class name: Atelectasis
ID: 14
279
class name: Calcification
ID: 3
960
class name: Cardiomegaly
ID: 0
5427
class name: Consolidation
ID: 14
556
class name: ILD
ID: 11
1000
class name: Infiltration
ID: 5
1247
class name: Lung Opacity
ID: 8
2483
class name: No finding
ID: 0
31818
class name: Nodule/Mass
ID: 13
2580
class name: Other lesion
ID: 7
2203
class name: Pleural effusion
ID: 13
2476
class name: Pleural thickening
ID: 14
4842
class name: Pneumothorax
ID: 14
226
class name: Pulmonary fibrosis
ID: 0
4655


Creo la clase neumonía

In [7]:
values = ['Consolidation', 'Infiltration']
df['Neumonia'] = 0
index = [i for i, clase in enumerate(df.class_name) if clase in values]
df['Neumonia'][index] = 1

/home/mr1142/miniconda3/envs/init/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,Neumonia
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580,0
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159,0
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336,0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880,0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072,0


Divido el dataset en dos y reduzco el de no neumonía

In [9]:
df_neumo = df[df.Neumonia == 1].reset_index(drop=True)
df_no_neumo = df[df.Neumonia == 0].reset_index(drop=True)

In [10]:
print(len(df_neumo))
print(len(df_no_neumo))

1803
66111


Para reducir elijo un numero de valores aleatorios que serán los índices

In [11]:
selection = np.random.randint(0, len(df_no_neumo), len(df_neumo))
df_no_neumo = df_no_neumo.iloc[selection,:]
len(df_no_neumo)

1803

Genero el nuevo dataframe juntando

In [12]:
df = pd.concat([df_neumo, df_no_neumo]).reset_index(drop=True)

Reordeno aleatoriamente

In [13]:
df = df.sample(frac=1)

In [14]:
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,Neumonia
501,20e27597c972c6e7fdb4d1e7638e227e,Consolidation,4,R8,1084.0,780.0,1636.0,2081.0,2304,2880,1
3048,793c5b4edfb03b51eb742cdf84702596,No finding,14,R13,NaN,NaN,NaN,NaN,2472,2739,0
539,0c31081e8ada2990bcbef0f12ea60b07,Infiltration,6,R8,188.0,1095.0,766.0,1540.0,1856,2160,1
79,0a072917005494298d153c01bbd8f689,Infiltration,6,R8,1144.0,1254.0,1617.0,1641.0,2048,2500,1
2122,d04f6328a4f309158aabe9818ab3e2eb,Cardiomegaly,3,R10,1066.0,1240.0,2038.0,1606.0,2540,3072,0


## Imagenes

In [15]:
def equalizar(img):
    # Contrast stretching
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    # Equalization
    img_eq = exposure.equalize_hist(img)
    # Adaptive Equalization
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)      
    return img_adapteq

In [16]:
def downsample(img):
    img = Image.fromarray(img)
    img = img.resize((512,512))
    img = np.array(img)
    img = img/255
    r = np.expand_dims(img, axis=-1)
    return r

In [17]:
def charge_img(name):
    print(f'\n {name}...')
    img = Image.open(os.path.join(path, 'train', name + '.png'))
    img = np.array(img)
    img = equalizar(img)
    img = downsample(img)
    print('charged')
    return(img)

In [18]:
tensor = np.zeros((len(df.image_id), 512, 512, 1))
for i in range(tensor.shape[0]):
    tensor[i,...] = charge_img(df.image_id[i])


 d7aea6f37597df566725da73655edcc2...
charged

 d77ef79ee4580a65dcb1e0e26b91d017...
charged

 eb6c714df22142229464c6b83e47d7d6...
charged

 bd3fe876153eeddad8bab49b129ea081...
charged

 038fc325ff4aa0a21c76d5dd7c740c89...
charged

 ca2a64d3c50654de6327f3880bbc17d3...
charged

 4b9ee23b2f17c3621158a9bef81cf86b...
charged

 0e6af94d17007c94d858b3bb7adb7dac...
charged

 7245fcab6a98659146972d983aa01c6d...
charged

 4869a847a5271f2eeffe65ff1db8a9b1...
charged

 e694841193437bcb89ad2c1acbf95f79...
charged

 7f5eab5de5307d4ae8e2a0fa99c0b41a...
charged

 62071e073320c21515baab6e5c55ddf8...
charged

 011ae9520e81f1efe71c9d954ec07d09...
charged

 24051b6c68ac93e887c70b671d9197cb...
charged

 665ba9b5664d7055c1d0ac1a44ea1ea9...
charged

 c37c1b0387090f95fa7f6b38542f1d19...
charged

 768480654fabe20d0c1340a17e129808...
charged

 3ee674202cc0887b7b43db270ca54555...
charged

 26585204e3c296a3b105bd5bd1c537ee...
charged

 5268b1ff61505e50ac05fd430cc45276...
charged

 aa91a19616d9ea16284536f902a58b92

charged

 8788745c121bcbce59873c943afa0ebd...
charged

 41cee81dbe9886a56b6c3ee56fb23448...
charged

 c60b87f681462b74b5cc58bc78f5b99e...
charged

 c2a2ce443b43b82ae5bd29d06716a44e...
charged

 eb319a3149253e25a32a6f2926cf67b2...
charged

 ba665972d68d2cb8b55839ad6592961c...
charged

 b065d3b4da92b8015b47224341661fd8...
charged

 fd1bd078f5f30ad02ed984c94077bbdb...
charged

 230ffa646a0bec6bce8b027cd4c66832...
charged

 846a659d3d5f0c9bc9a7f917dea9da79...
charged

 e2de831b92a2dc7237d2b1885ee4bb9c...
charged

 807797c8ef120370d45796ed35260d81...
charged

 a9141690efddf683c82a9d90af347ab1...
charged

 6d6746496137a10dbfbb4019c47174c0...
charged

 d6352ccc0bf75017b99820231822ab6b...
charged

 1aaa4b217affae30113bd3a7a384a4c7...
charged

 ce2c5376b597eb067395da4c7f9960aa...
charged

 943587a46ada0ee193c60a106ac61934...
charged

 f59ccb89d776a68b79292bef810333ac...
charged

 784a4ee98cfc37a7c1ae9721bc8603ed...
charged

 3cdce1131cdbbbc959330bb08be55a3d...
charged

 a3106645d9ff642b102c1fae

charged

 82877be2465c084b0b9bc186fc7f158f...
charged

 a593d4aa6d8d9dbe2139c6c3889535eb...
charged

 179caec60f7bf3c620c28350354861a1...
charged

 cdfac35a0800774db5fe8a6354df64ef...
charged

 230ffa646a0bec6bce8b027cd4c66832...
charged

 064023f1ff95962a1eee46b9f05f7309...
charged

 1ec5b9d1af1a295a5e0d1da32d4ed835...
charged

 1c1a57cb8b3893348ecfb6e704a1b87e...
charged

 92e9e7ccc49692f0c6fa6917d7289f6f...
charged

 3c0343764fd0e6b3edb40b15740a597e...
charged

 2f111f79e145a2062200644c1e77dbd4...
charged

 5a58e4e711cccb78407ae86820d643d0...
charged

 697c98dcc1457ad774db7b42a4e1e8b4...
charged

 4db4ecf3e2d32c92cf3f10c7b1643a5b...
charged

 0005e8e3701dfb1dd93d53e2ff537b6e...
charged

 4f3f5460ae80ab6ec1bf00b33087a828...
charged

 c723c6ffb2f746dff02c9538c6c593f4...
charged

 764eeaca1e91f7410e68f3daf40de6ab...
charged

 a2d8804d3d08a2afb75dd9e6fbb53010...
charged

 82d5f09eac7d3abea784450255fddc78...
charged

 1b0adf573618b9d4c94b1890852179a0...
charged

 66cd940a7bbf8b37c6abecc0

charged

 d8b297c773294eb23707c0a7f693c5fc...
charged

 6cf0ea4eb8810156bfb48f5a7dbeffd3...
charged

 5161ae032f0552603b6de965b2bb3e09...
charged

 17d5b60d0b5decddb51eba55dc4a514b...
charged

 4b001bab36d94f73c1ead3ab74690dbc...
charged

 046ad82db74a4155eaff3586b0d1e068...
charged

 1c2edaf3f7c1867f34d76dafe8f53d80...
charged

 45f8fd11d471cfb910172d26f8157ee5...
charged

 4b8c24a1eaeabd898c28a92d25cc3632...
charged

 15f023463d88b8f8bc852a9a7b72e6e3...
charged

 1fce42f55a0de66eb161fc831cd33cc3...
charged

 c2fafca7a6e8316c8bc0a3dc400b8aba...
charged

 7e9efb8ee0bae7af280f5ea091f8d245...
charged

 2895afe060760a377fea0d6c8fcbf9d1...
charged

 2546f825d7846960568c710eab2cbbd3...
charged

 3cac0eba7aff674db58e5e7d5cebe985...
charged

 82f62718d5385a45eb2b328b3a90fbdd...
charged

 1ed4cf45940bbfbcdc31cf4289c1c1e4...
charged

 5bb68aea19f6eb00d28d0e6386afecfc...
charged

 7da8cf1d4261791fbb0bae6f6bdb33ec...
charged

 e9581123b6819b2cd1bcf6ed35481520...
charged

 fa7d454b6cb43448ed5a8da4

charged

 89e6ab133f587191383608ee04cea79a...
charged

 dad083d8150288db33157fde49ab35f9...
charged

 ba5e3409250a85483d6e39be759bc102...
charged

 ca300a7c90492a6f03220a3f9efe7c4e...
charged

 a75ffc1c7b9a233ab869e52d79cefefb...
charged

 363999879bc614085397f192124c23b0...
charged

 924eb4734344e5808df5d15855d68178...
charged

 f2e56bf4b304000674dde626b3335ca7...
charged

 78e8eee238eeb6f386be08ece092167c...
charged

 85cfc59f138f490784170dbaeb0112a7...
charged

 fc50039c45fdb6c9224bfff5ba4e64b3...
charged

 064023f1ff95962a1eee46b9f05f7309...
charged

 3ee674202cc0887b7b43db270ca54555...
charged

 89e503ac5a54714d03b313b1a6fac252...
charged

 7a99127d0a0c6a00aad2115a6ff9e653...
charged

 fc50039c45fdb6c9224bfff5ba4e64b3...
charged

 f5824499552ae516c70308416344eb91...
charged

 fb65f00dc9ea6fe17670ad9564e41db5...
charged

 662fe30af3399b56c1579e95b0c6c87e...
charged

 92928a912d1736877050c8c00c3dacdb...
charged

 2a3def0aa2b27bea4235348e5d4cf345...
charged

 d7c148eb16f56462447f7f9c

...

charged

 8751da44b525b4328e1457adab72f8e0...
charged

 41c5464286501600f3a4c2f7a5c4c9e3...
charged

 8314e6597bd397bb4a5836c042499b27...
charged

 8fcbf4c0ba3178b938cc3c80df975fe3...
charged

 a838e79ba2e9716bc790a76f7ae1c94e...
charged

 300c691ccde48d55c40f93dd9cac64fa...
charged

 62af91be77e451f474fd867b3e6bbae3...
charged

 bf51e1b66e96e5276e009290ab020dc3...
charged

 4361c97e29d8614a2b2a539beaf8aef0...
charged

 dfd523a5991fc852654bf1235c6282c6...
charged

 604340ab6423f81a91ac72182163e9a4...
charged

 ba46dcb445340df33566b52d7192ab6e...
charged

 ce4e8ead795a19444e7b9dc1c27cfb66...
charged

 5ca7d0473877e6d03d94a171d55cec89...
charged

 0b62bc6644be72ce4dfa5ea77a77f311...
charged

 d77de97b5f240fd0fcebdb486062467b...
charged

 bab58da5c7d82e57bd63dfa1d6e698a4...
charged

 b54b56b472db108351ef8c82ee619806...
charged

 eda50be538fe17c4f5e36050a8fbcc86...
charged

 088869b047b7336c8d31bea5cf10cf24...
charged

 b8b6dd22e9ab633792b6d17cb2f254a3...
charged

 bb541cb8a793063968903bf3

charged

 10e7dac260f11046164aa94f2d8add9f...
charged

 c39440999e8b3e376911ab851a7c56f6...
charged

 8a6d6cfef94546b6833aca6e9acaa406...
charged

 1283d5ba97663aef3b97ee4f1884e9f9...
charged

 9607e2f142ad3dbaa2ba8db48dc81daf...
charged

 3fbf33a57cf10875ba8913ce4ba40a56...
charged

 a6be2e43de24f6b2d0b1f908f181cbe5...
charged

 81545a6a71bbc9cb3d50ef6ba9ccaeaf...
charged

 8e5f9d775ab631aca970ca98d32946dc...
charged

 08e3d3783927a15d35137c756c1a7e0b...
charged

 039c6997248b3a6ba3c909af35fce0eb...
charged

 e76af44f2856bf89d69d00d43536cf8c...
charged

 3105bf6d00f6c2164ea9285b20692df5...
charged

 10fa1e4bf25d6d057c88210c3a1fc645...
charged

 8bbf80bd095c75b2207ef57a277a661d...
charged

 23b5a0775b57978b873274fd8e155ffb...
charged

 c89fdbd26b69a96edbef96f88a4004a5...
charged

 57415c4992802274b5a54930019c30f1...
charged

 be97043265be8c30ef57f278b3ca24b2...
charged

 ded121c1fb2afcda23e7eb5aefa7daa0...
charged

 6e2c5ac0b017ee374840d5ad3aba3629...
charged

 fe8f22bba9ea2e1421431805

charged

 bbc8cca2bda1c6bc5b66715a12830d78...
charged

 cf32a2bc41deeb7436a36bcc4ff324fb...
charged

 c2209aca8dc71cfeba0ac0f9ad5f0f45...
charged

 d8bd080261025141ad5aec15b3b6786d...
charged

 d5203efda6708309bae1c60534f16192...
charged

 f32fbc404c68b727c5bf14bca3ec7b5e...
charged

 b9f09915187c2cc01e958da90d97ac89...
charged

 d088c22989ec259a26d64efe51eb276d...
charged

 98dca7e62a4f2d47455b9a275fa75a08...
charged

 ab99fbfc6bf9833e2a064e8b479af0b3...
charged

 1b3f4c30e797e1f54159d2cb01a60738...
charged

 e345cc1bc62b3ad7f37e7fbb540311f0...
charged

 305e4add9c72c91e9984305bf4e85aee...
charged

 bf5c7a815313c2d2853494cf766237b5...
charged

 86ea362edec93fb69697ad8e6920d0d7...
charged

 89e503ac5a54714d03b313b1a6fac252...
charged

 93b7bc8659a0dd1ef99a00fd13db114d...
charged

 7cd744553edddae9915bc1d28fe5831a...
charged

 c5f28e4cb80cb30289e4dc4f79304dfa...
charged

 98553a08561599e00ebcbc603e206557...
charged

 c42997b00e59f4523788aa9fbe1f7526...
charged

 54e6184c63c75a9695d7effc

charged

 fd6d801a9b661fd0abe97e5c37567c2b...
charged

 9278f10a8ff708a2cb687a2f0366ad74...
charged

 66a42146929520fe337cad37816f4384...
charged

 6544852e523b70450c51b767f71eb275...
charged

 96331146d9845ecb8a6ae3d15e3bf7bf...
charged

 2b82db51c5129aabadccfe4738b596b2...
charged

 05ca9b3be8bc8240cef9a387a5a06785...
charged

 711f2dc8c252adfdfc30cd080c23699b...
charged

 9d25a49dcfd33a593605da5b2cecfe2b...
charged

 e2b6dcfadc876caf1673923ee3cedf7d...
charged

 85e4008def6c48bf43bcf70a5d1e2273...
charged

 430e3d75b2b3924e82d5fcf9bec84e8b...
charged

 264c01b040113c32f5d50f40b91d4061...
charged

 a2ca9713ed2b4fb5632aa8647769c1a6...
charged

 7bd83b3d3af0ea8150c8f2a737712db1...
charged

 38ab2d3283b496d60dc137e94d16b7c6...
charged

 051c2436c0acdb5e09d085c7e4a764f3...
charged

 8c6cbbc156f7eb5d6132c937d089f014...
charged

 7d90de12500e159955637cd98bf0e87c...
charged

 481c6d677933eac40f7bbdc2933b0135...
charged

 3bf462b3f48c0a1c35912a77a84f318d...
charged

 a2c6dd0cabc2fc70fbb1cf2d

charged

 e4a73f3555433a101aa6f6a0ca5006ff...
charged

 7253800842122c9e6e95878b46008f54...
charged

 f7a56f47ad3ca3e1a06fd41bba7a3b03...
charged

 e9954e6e3b2d0c5bf990a519c0ba5abe...
charged

 6b1629617f13b37b41af2c15243bf277...
charged

 16531c511f60bb26a0f3e36acf16b73a...
charged

 d9b51a0471d1ea9b698c9a88c7888925...
charged

 41ebb4e4b4434bb495434e6a4026c9d7...
charged

 b35b40fca325eb0e9e5d699e6ef55d4b...
charged

 d312cdd620b130479bfea6128e47b2c4...
charged

 dd4895fd4a9375acfc43f2f9a51a8d49...
charged

 996589592a0f854d6aeefa817f76ffde...
charged

 45dd8541d7346dd9015404cadcc1a8f8...
charged

 73ceb7e9d4f1e6737ae3e1b905bec1c1...
charged

 a61be8051f02ec494ed40696e988f6d1...
charged

 5d3d38eae35191d06fd2a0261fa74934...
charged

 29491e5d92ee6968b5dc51fdab1f95ed...
charged

 70fecc68e1f378ef9519591a3cce762d...
charged

 84e9f232df2236e9dc7f7f472ff26bee...
charged

 384f1bceb95a5f2dddcd70e000a5051d...
charged

 a2ca3716b1e7923e5f8e0fc6f8200509...
charged

 153da4dfa81a43cdf1176f78

## X e Y

In [19]:
Y = np.array(df['Neumonia'])
X = tensor

In [20]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.2, 
                                                                    random_state=42, shuffle=True, stratify=Y )

# Modelo

- Añado una capa inicial para poder entregarle el input de 3 canales. 
- Añado 1 convolución con max pooling y una segunda convolución cno globalMaxPooling para ajustar mejor
- Añado Dropout para evitar overfiting
- Añado la capa final con 6 neuronas de salida

In [21]:
input_shape = (512,512,3)
conv_base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)

In [22]:
len(conv_base.layers)

237

In [23]:
len(conv_base.trainable_variables)

211

In [24]:
model = models.Sequential()
model.add(layers.Conv2D(3,3,padding="same", input_shape=(512,512,1), activation='elu', name = 'conv_inicial'))
model.add(conv_base)
model.add(layers.Conv2D(3,32, padding='same', input_shape=(8,8,1280), activation='selu', name = 'conv_posterior'))
model.add(layers.MaxPool2D(pool_size = (2,2), padding='same', name = 'first_pooling'))
model.add(layers.Conv2D(3,64, padding='same', input_shape=(4,4,1280), activation='selu', name = 'last_convolution'))
model.add(layers.GlobalMaxPooling2D(name="general_max_pooling"))
model.add(layers.Dropout(0.2, name="dropout_out"))
model.add(layers.Dense(1, activation="softmax", name="fc_out"))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_inicial (Conv2D)        (None, 512, 512, 3)       30        
_________________________________________________________________
efficientnetb0 (Functional)  (None, 16, 16, 1280)      4049571   
_________________________________________________________________
conv_posterior (Conv2D)      (None, 16, 16, 3)         3932163   
_________________________________________________________________
first_pooling (MaxPooling2D) (None, 8, 8, 3)           0         
_________________________________________________________________
last_convolution (Conv2D)    (None, 8, 8, 3)           36867     
_________________________________________________________________
general_max_pooling (GlobalM (None, 3)                 0         
_________________________________________________________________
dropout_out (Dropout)        (None, 3)                 0

# Compilación y entrenamiento

## 1º entrenamiento
Solo vamos a entrenar las capas que hemos añadido

In [26]:
conv_base.trainable = False

In [27]:
len(model.trainable_variables)

8

### Hiperparámetros

1. **Loss**

In [28]:
loss = tf.keras.losses.BinaryCrossentropy()

2. **Optimizador y learning rate**

In [29]:
lr = 0.01
opt = tf.keras.optimizers.Adam(learning_rate = lr)

3. **Metricas**

In [30]:
met = ['CategoricalAccuracy', 'AUC']

4. **Batch size**

In [31]:
batch = 32

5. **Epochs**

In [32]:
initial_epochs = 30

### Entrenamiento

In [33]:
model.compile(optimizer=opt, loss = loss , metrics = met)

In [34]:
print('loss, acuracy, auc')
model.evaluate(X_test, Y_test)

loss, acuracy, auc
23/23 [==============================] - 285s 6s/step - loss: 0.6953 - categorical_accuracy: 1.0000 - auc: 0.5000


[0.694256067276001, 1.0, 0.5]

In [ ]:
history = model.fit(X_train,Y_train, batch_size = batch, epochs = initial_epochs, 
                    validation_data = (X_test, Y_test))

Epoch 1/30


In [ ]:
epochs_range = range(initial_epochs)
i = 0
for i in range(int(len(history.history.keys())/2)):
    key = list(history.history.keys())[i]
    
    plt.figure(figsize=(5, 5))
    plt.subplot(1, 1, 1)
    plt.plot(epochs_range, history.history[key], label=key)
    plt.plot(epochs_range, history.history['val_' + key], label='val_' + key)
    plt.legend(loc='lower right')
    plt.title('Training and Validation')
plt.show()

## 2º Entrenamiento
Ahora vamos a entrenar tambien la red preentrenada, pero no queremos entrenar todas las capas, únicamente las capas más profundas, ya que las primeras capas son muy genéricas.

### Ajustar el entrenamiento de la red preentrenada

In [ ]:
conv_base.trainable = True

In [ ]:
len(conv_base.layers)

Vamos a entrenar desde la capa 100

In [ ]:
fine_tune_at = 100

for layer in conv_base.layers[:fine_tune_at]:
    layer.trainable = False

## Hiperparámetros
Como el modelo es mucho más grande tenemos que usar un learning rate más bajo del que usabamos.
Vamos a utilizar dos learning rates, uno para el modelo preentrenado y otro para las capas nuevas

In [ ]:
opt = [tf.keras.optimizers.Adam(learning_rate = lr), tf.keras.optimizers.RMSprop(learning_rate = lr/100)]
optimizers_and_layers = [(opt[0], model.layers[0]),
                         (opt[1], model.layers[1]),
                         (opt[0], model.layers[2:])]
opt = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

In [ ]:
batch_size = 32
fine_tune_epochs = 50
total_epoch = initial_epochs + fine_tune_epochs

### Entrenamiento

In [ ]:
model.compile(optimizer=opt, loss = loss , metrics = met)

In [ ]:
history = model.fit(X_train,Y_train, 
                    batch_size = batch_size,
                    epochs = total_epoch,
                    initial_epoch=history.epoch[-1],
                    validation_data = (X_test, Y_test))

In [ ]:
epochs_range = len(history.history['loss'])
i = 0
for i in range(int(len(history.history.keys())/2)):
    key = list(history.history.keys())[i]
    
    plt.figure(figsize=(5, 5))
    plt.subplot(1, 1, 1)
    plt.plot(history.history[key], label=key)
    plt.plot(history.history['val_' + key], label='val_' + key)
    plt.legend(loc='lower right')
    plt.title('Training and Validation')
plt.show()